3種類のアンサンブル学習をスクラッチ実装していきます。そして、それぞれの効果を小さめのデータセットで確認します。


・ブレンディング

・バギング

・スタッキング

小さなデータセットの用意

以前も利用した回帰のデータセットを用意します。


[House Prices: Advanced Regression Techniques](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)


この中のtrain.csvをダウンロードし、目的変数としてSalePrice、説明変数として、GrLivAreaとYearBuiltを使います。


train.csvを学習用（train）8割、検証用（val）2割に分割してください。

scikit-learn

単一のモデルはスクラッチ実装ではなく、scikit-learnなどのライブラリの使用を推奨します。


[sklearn.linear_model.LinearRegression — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)


[sklearn.svm.SVR — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html)


[sklearn.tree.DecisionTreeRegressor — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html)

#ブレンディング

##ブレンディングとは

ブレンディングとは、N個の多様なモデルを独立して学習させ、推定結果を重み付けした上で足し合わせる方法です。最も単純には平均をとります。多様なモデルとは、以下のような条件を変化させることで作り出すものです。


・手法（例：線形回帰、SVM、決定木、ニューラルネットワークなど）

・ハイパーパラメータ（例：SVMのカーネルの種類、重みの初期値など）

・入力データの前処理の仕方（例：標準化、対数変換、PCAなど）

重要なのはそれぞれのモデルが大きく異なることです。


回帰問題でのブレンディングは非常に単純であるため、scikit-learnには用意されていません。


《補足》


分類問題の場合は、多数決を行います。回帰問題に比べると複雑なため、scikit-learnにはVotingClassifierが用意されています。


[sklearn.ensemble.VotingClassifier — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.VotingClassifier.html)



In [475]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor


In [476]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [477]:
data = pd.read_csv('/content/drive/My Drive/House Prices_train.csv')
data

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,...,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,...,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,...,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,...,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,...,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,...,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1999,2000,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,Unf,0,Unf,0,953,953,GasA,...,Y,SBrkr,953,694,0,1647,0,0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1999.0,RFn,2,460,TA,TA,Y,0,40,0,0,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,6,6,1978,1988,Gable,CompShg,Plywood,Plywood,Stone,119.0,TA,TA,CBlock,Gd,TA,No,ALQ,790,Rec,163,589,1542,GasA,...,Y,SBrkr,2073,0,0,2073,1,0,2,0,3,1,TA,7,Min1,2,TA,Attchd,1978.0,Unf,2,500,TA,TA,Y,349,0,0,0,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,9,1941,2006,Gable,CompShg,CemntBd,CmentBd,None,0.0,Ex,Gd,Stone,TA,Gd,No,GLQ,275,Unf,0,877,1152,GasA,...,Y,SBrkr,1188,1152,0,2340,0,0,2,0,4,1,Gd,9,Typ,2,Gd,Attchd,1941.0,RFn,1,252,TA,TA,Y,0,60,0,0,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,6,1950,1996,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,TA,TA,Mn,GLQ,49,Rec,1029,0,1078,G

In [478]:
X = np.array(data[["GrLivArea","YearBuilt"]])
y = np.array(data['SalePrice']).reshape(-1,)

##【問題1】ブレンディングのスクラッチ実装

ブレンディング をスクラッチ実装し、単一モデルより精度があがる例を 最低3つ 示してください。精度があがるとは、検証用データに対する平均二乗誤差（MSE）が小さくなることを指します。


In [479]:
lr = LinearRegression()
svr = SVR()
dtr = DecisionTreeRegressor()

単一モデルの場合(SVR使用)

In [480]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=100)
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(1168, 2)
(292, 2)
(1168,)
(292,)


In [481]:
#標準化、対数変換
ss = StandardScaler()
ss.fit(X_train)
X_train = ss.transform(X_train)
ss.fit(X_val)
X_val = ss.transform(X_val)

In [482]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred1 = lr.predict(X_val)
print ("線形回帰のMSE: {:.4f}".format(np.log(mean_squared_error(y_val, y_pred1))))

線形回帰のMSE: 21.3863


In [483]:
svr = SVR()
svr.fit(X_train, y_train)
y_pred2 = svr.predict(X_val)
print ("SVRのMSE: {:.4f}".format(np.log(mean_squared_error(y_val, y_pred2))))

SVRのMSE: 22.6466


In [484]:
dtr = DecisionTreeRegressor()
dtr.fit(X_train,y_train)
y_pred3 = dtr.predict(X_val)
print ("決定木のMSE: {:.4f}".format(np.log(mean_squared_error(y_val, y_pred3))))

決定木のMSE: 21.6699


In [485]:
def blending(y_val, pred1, pred2, pred3):
    pred_all = (pred1 + pred2 + pred3) / 3
    return print("ブレンディングのMSE: {:.4f}".format(np.log(mean_squared_error(y_val, pred_all))))

In [486]:
blending(y_val, y_pred1, y_pred2, y_pred3)

ブレンディングのMSE: 21.5369


ブレンディングの場合

・決定木パラメーター調整

In [487]:
dtrt = DecisionTreeRegressor(max_depth=6)# max_depth=6がベスト
dtrt.fit(X_train,y_train)
yn_pred3 = dtrt.predict(X_val)
print ("決定木のMSE: {:.4f}".format(np.log(mean_squared_error(y_val, yn_pred3))))

決定木のMSE: 21.2897


精度向上

・sklearnのPipelineとGridSearchCVを使用し、PCAのパラメーターとSVRのパラメーターをいっぺんにグリッドサーチしてみよう



In [488]:
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

pl1 = Pipeline([['pca', PCA(random_state=826)],
                ['svr', SVR(kernel='linear')]])

In [489]:
prms1 = {'pca__n_components': [0,0.1, 0.5, 0.9],
         'svr__C': [0,0.1, 0.5, 1.],
         'svr__epsilon': [0,0.05, 0.10, 0.20]}

In [490]:
from sklearn.model_selection import GridSearchCV
gs1 = GridSearchCV(pl1, prms1, n_jobs=-1, return_train_score=True, cv=5, verbose=10)
gs1.fit(X_train, y_train)


Fitting 5 folds for each of 64 candidates, totalling 320 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0062s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0062s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0230s.) Setting batch_size=8.
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0400s.) Setting batch_size=16.
[Parallel(n_jobs=-1)]: Done  36 tasks      | elapsed:    0.2s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0786s.) Setting batch_size=32.
[Parallel(n_jobs=-1)]: Done 100 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Batch computation too slow (2.3423s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Batch computation too slow (2.1967s.) Setting batch_size=1.
[Parallel(n_jobs=-1)]: Done 221 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Batch co

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[['pca',
                                        PCA(copy=True, iterated_power='auto',
                                            n_components=None, random_state=826,
                                            svd_solver='auto', tol=0.0,
                                            whiten=False)],
                                       ['svr',
                                        SVR(C=1.0, cache_size=200, coef0=0.0,
                                            degree=3, epsilon=0.1,
                                            gamma='scale', kernel='linear',
                                            max_iter=-1, shrinking=True,
                                            tol=0.001, verbose=False)]],
                                verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'pca__n_components': [0, 0.1, 0.5, 0.9],
           

In [491]:
print ("SVRのMSE: {:.4f}".format(np.log(mean_squared_error(gs1.predict(X_val),y_val))))

SVRのMSE: 22.6286


0.2下がったがほとんど差がない

PCA使用しないSVRは？

In [492]:
prms2 = {'C': [0,0.1, 0.5, 1.],
        'epsilon': [0,0.05, 0.10, 0.20]}
gs2 = GridSearchCV(SVR(kernel='linear'), prms2, n_jobs=-1, return_train_score=True, cv=5, verbose=10)
gs2.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0050s.) Setting batch_size=2.
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0108s.) Setting batch_size=4.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Batch computation too fast (0.0131s.) Setting batch_size=8.


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:    2.5s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='linear',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0, 0.1, 0.5, 1.0],
                         'epsilon': [0, 0.05, 0.1, 0.2]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring=None, verbose=10)

In [493]:
print ("SVRのMSE: {:.4f}".format(np.log(mean_squared_error(gs2.predict(X_val),y_val))))

SVRのMSE: 22.6286


PCAの有無での差はないが、グリッドサーチを使用すると多少数値が良くなる



In [494]:
#モデルにfor文で重みを調整する

def blend(pred1,pred2,pred3,y_val):

    y3= 100
    best_param={}
    #重み計算
    for x1 in np.arange(0.1, 2.0, 0.2):
        for x2 in np.arange(0.1, 2.0, 0.2):
            for x3 in np.arange(0.1, 2.0, 0.2):
                #モデルに重みかける
                y_pred_all = (y_pred1 * x1 + y_pred2 * x2 + y_pred3 * x3) / 3
                # MSE
                mse = (np.log(mean_squared_error(y_val, y_pred_all)))
                best_param[mse] = [x1, x2, x3]
                y3 = min(y3, mse)

    print("重み調整のブレンディングのMSE: {:.4f}".format(y3)) # MSE
    print("ベストパラメーター: {}".format(best_param[y3])) #もっとも値の良い重みを出している

In [495]:
blend(y_pred1,y_pred2,y_pred3,y_val)

重み調整のブレンディングのMSE: 21.2799
ベストパラメーター: [1.9000000000000004, 0.1, 1.1000000000000003]


In [496]:
blending(y_val, y_pred1, y_pred2, y_pred3)
blend(y_pred1, y_pred2, yn_pred3, y_val)

ブレンディングのMSE: 21.5369
重み調整のブレンディングのMSE: 21.2799
ベストパラメーター: [1.9000000000000004, 0.1, 1.1000000000000003]


精度向上　大・中・小

1.決定木パラメーター調整　大

2.グリッドサーチ&PCA　小

3.重み調整　大

##バギングとは

バギングは入力データの選び方を多様化する方法です。学習データから重複を許した上でランダムに抜き出すことで、N種類のサブセット（ ブートストラップサンプル ）を作り出します。それらによってモデルをN個学習し、推定結果の平均をとります。ブレンディングと異なり、それぞれの重み付けを変えることはありません。


[sklearn.model_selection.train_test_split — scikit-learn 0.21.3 documentation](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)


scikit-learnのtrain_test_splitを、shuffleパラメータをTrueにして使うことで、ランダムにデータを分割することができます。これによりブートストラップサンプルが手に入ります。


推定結果の平均をとる部分はブースティングと同様の実装になります。

##【問題2】バギングのスクラッチ実装

バギング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

単一の場合

In [497]:
X_train1, X_val1, y_train1, y_val1 = train_test_split(X, y, test_size=0.2, random_state = 0)
print(X_train1.shape)
print(X_val1.shape)
print(y_train1.shape)
print(y_val1.shape)

(1168, 2)
(292, 2)
(1168,)
(292,)


In [498]:
lr2 = LinearRegression()
lr2.fit(X_train1,y_train1)
yb_pred1 = lr2.predict(X_val1)
mse = (np.log(mean_squared_error(y_val1, yb_pred1)))
print("線形回帰のMSE :{:.2f}".format(mse))

線形回帰のMSE :21.80


In [499]:
svr2 = SVR()
svr2.fit(X_train1,y_train1)
yb_pred2 = svr2.predict(X_val1)
mse = (np.log(mean_squared_error(y_val1, yb_pred2)))
print("SVRのMSE :{:.2f}".format(mse))

SVRのMSE :22.70


In [500]:
dtr2 = DecisionTreeRegressor()
dtr2.fit(X_train1,y_train1)
yb_pred3 = dtr2.predict(X_val1)
mse = (np.log(mean_squared_error(y_val1, yb_pred3)))
print("決定木のMSE :{:.2f}".format(mse))

決定木のMSE :21.91


バギングのスクラッチ

In [501]:
def bagging(X, y, model, k):

    np.random.seed(0)
    # model判別
    if model == "LinearRegression":
        model = LinearRegression()
    elif model == "SVR":
        model = SVR()
    elif model == "DecisionTreeRegressor":
        model = DecisionTreeRegressor(random_state=0)
    
    for i in range(k):
        #ブートストラップサンプル
        X_train2, X_val2, y_train2, y_val2 = train_test_split(X, y, shuffle=True, test_size=0.5, random_state = None)
        model.fit(X_train2, y_train2)
        pred = model.predict(X_val2)
    mse = (np.log(mean_squared_error(y_val2, pred)))
    return print("MSE :{:.2f}".format(mse))

In [502]:
#lr
bagging(X,y,"LinearRegression",3)
#svr
bagging(X,y,"SVR",3)
#dtr
bagging(X,y,"DecisionTreeRegressor",3)

MSE :21.54
MSE :22.48
MSE :21.63


MSE向上

##スタッキングとは

スタッキングの手順は以下の通りです。最低限ステージ0とステージ1があればスタッキングは成立するため、それを実装してください。まずは $K_0=3, M_0=2$ 程度にします。


《学習時》


（ステージ $0$ ）


・学習データを $K_0$ 個に分割する。

・分割した内の $(K_0 - 1)$ 個をまとめて学習用データ、残り $1$ 個を推定用データとする組み合わせが $K_0$ 個作れる。

・あるモデルのインスタンスを $K_0$ 個用意し、異なる学習用データを使い学習する。
・それぞれの学習済みモデルに対して、使っていない残り $1$ 個の推定用データを入力し、推定値を得る。（これをブレンドデータと呼ぶ）

・さらに、異なるモデルのインスタンスも $K_0$ 個用意し、同様のことを行う。モデルが $M_0$ 個あれば、 $M_0$ 個のブレンドデータが得られる。

（ステージ $n$ ）


・ステージ $n-1$ のブレンドデータを$M_{n-1}$ 次元の特徴量を持つ学習用データと考え、 $K_n$ 個に分割する。以下同様である。

（ステージ $N$ ）＊最後のステージ


・ステージ $N-1$ の $M_{N-1}$ 個のブレンドデータを$M_{N-1}$ 次元の特徴量の入力として、1種類のモデルの学習を行う。これが最終的な推定を行うモデルとなる。

《推定時》


（ステージ $0$ ）


・テストデータを $K_0×M_0$ 個の学習済みモデルに入力し、$K_0×M_0$ 個の推定値を得る。これを $K_0$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $n$ ）


・ステージ $n-1$ で得たブレンドテストを $K_n×M_n$ 個の学習済みモデルに入力し、$K_n×M_n$ 個の推定値を得る。これを $K_n$ の軸で平均値を求め $M_0$ 次元の特徴量を持つデータを得る。（ブレンドテストと呼ぶ）

（ステージ $N$ ）＊最後のステージ


・ステージ $N-1$ で得たブレンドテストを学習済みモデルに入力し、推定値を得る。

##【問題3】スタッキングのスクラッチ実装

スタッキング をスクラッチ実装し、単一モデルより精度があがる例を 最低1つ 示してください。

[参考](https://rin-effort.com/2020/02/09/learn-stacking/#toc2)

In [503]:
X_train3, X_val3, y_train3, y_val3 = train_test_split(X, y, test_size=0.2, random_state = 0)
print(X_train3.shape)
print(X_val3.shape)
print(y_train3.shape)
print(y_val3.shape)

(1168, 2)
(292, 2)
(1168,)
(292,)


単一の場合

In [504]:
lr3 = LinearRegression()
lr3.fit(X_train3,y_train3)
t_pred1 = lr3.predict(X_val3)
mse = (np.log(mean_squared_error(y_val3, t_pred1)))
print("決定木のMSE :{:.2f}".format(mse))

決定木のMSE :21.80


In [505]:
svr3 =SVR()
svr3.fit(X_train3,y_train3)
t_pred2 = svr3.predict(X_val3)
mse = (np.log(mean_squared_error(y_val3, t_pred2)))
print("決定木のMSE :{:.2f}".format(mse))

決定木のMSE :22.70


In [506]:
dtr3 = DecisionTreeRegressor()
dtr3.fit(X_train3,y_train3)
t_pred3 = dtr3.predict(X_val3)
mse = (np.log(mean_squared_error(y_val3, t_pred3)))
print("決定木のMSE :{:.2f}".format(mse))

決定木のMSE :21.90


In [507]:
def test(y_val, pred1, pred2, pred3):
    pred_all = (pred1 + pred2 + pred3) / 3
    return print("スタッキングのMSE: {:.4f}".format(np.log(mean_squared_error(y_val, pred_all))))

test(y_val3, t_pred1, t_pred2, t_pred3)

スタッキングのMSE: 21.7483


スタッキングの場合

[参考](http://segafreder.hatenablog.com/entry/2016/05/24/235822)

In [508]:
X_train = data[["GrLivArea","YearBuilt"]]
y_train = data["SalePrice"]
X_val = data[["GrLivArea","YearBuilt"]]

In [509]:
from sklearn.model_selection import KFold

In [510]:
def stacking(model, train_x, train_y, test_x):
    preds = []
    preds_test = []
    va_idxes = []

    kf = KFold(n_splits=4, shuffle=True, random_state=71)

    # クロスバリデーションで学習,予測し、予測値とインデックスを保存する
    for i, (tr_idx, va_idx) in enumerate(kf.split(train_x)):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        model.fit(tr_x, tr_y, va_x, va_y)
        pred = model.predict(va_x)
        preds.append(pred)
        pred_test = model.predict(test_x)
        preds_test.append(pred_test)
        va_idxes.append(va_idx)

    # バリデーションデータに対する予測値を連結し、その後元の順序に並べ直す
    va_idxes = np.concatenate(va_idxes)
    preds = np.concatenate(preds, axis=0)
    #配列のインデックス返す
    order = np.argsort(va_idxes)
    #print(order)
    pred_train = preds[order]

    # テストデータに対する予測値の平均をとる
    preds_test = np.mean(preds_test, axis=0)

    return pred_train, preds_test


学習する時 fit,予測するときはpredictが統一できるので便利

In [511]:
#lr
class Model1LR:
    def __init__(self):
        self.model = None
        self.scaler = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        self.model = LinearRegression()
        self.model.fit(tr_x, tr_y)

    def predict(self, x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

In [512]:
#SVR
class Model1SVR:
    def __init__(self):
        self.model = None
        self.scaler = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        self.model =SVR()
        self.model.fit(tr_x, tr_y)

    def predict(self, x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

In [513]:
#TDR
class Model1DTR:
    def __init__(self):
        self.model = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        self.model = DecisionTreeRegressor(
            max_depth=5,
            random_state=10,
        )
        self.model.fit(tr_x,tr_y)

    def predict(self,x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

2層目は線形回帰で設定

In [514]:
class Model2LR:
    def __init__(self):
        self.model = None
        self.scaler = None

    def fit(self, tr_x, tr_y, va_x, va_y):
        self.scaler = StandardScaler()
        self.scaler.fit(tr_x)
        tr_x = self.scaler.transform(tr_x)
        self.model = LinearRegression()
        self.model.fit(tr_x, tr_y)

    def predict(self, x):
        x = self.scaler.transform(x)
        pred = self.model.predict(x)
        return pred

1層目のモデル

In [515]:
model1_lr = Model1LR()
pred_train_lr, pred_test_lr = stacking(model1_lr, X_train , y_train, X_val)

model1_svr = Model1SVR()
pred_train_svr, pred_test_svr = stacking(model1_svr, X_train , y_train, X_val)

model1_TDR = Model1DTR()
pred_train_dtr, pred_test_dtr = stacking(model1_TDR, X_train , y_train, X_val)

1層目のモデルの評価

In [516]:
print ("lrのMSE: {:.2f}".format(np.log(mean_squared_error(y_train, pred_train_lr))))
print ("SVRのMSE: {:.2f}".format(np.log(mean_squared_error(y_train, pred_train_svr))))
print ("決定木のMSE: {:.2f}".format(np.log(mean_squared_error(y_train, pred_train_dtr))))

lrのMSE: 21.51
SVRのMSE: 22.61
決定木のMSE: 21.42


この段階でMSE向上

予測値を特徴量としてデータフレームを作成

In [517]:
X_train2 = pd.DataFrame({'pred_train_lr': pred_train_lr, 'pred_train_svr': pred_train_svr, 'pred_train_dtr': pred_train_dtr})
X_val2 = pd.DataFrame({'pred_test_lr': pred_test_lr, 'pred_test_svr': pred_test_svr, 'pred_test_dtr': pred_test_dtr})

X_train2.head(3)

,pred_train_lr,pred_train_svr,pred_train_dtr
0,230761.760789,163284.856978,211789.552239
1,161713.259950,164048.515408,150220.306452
2,235811.138074,163292.783025,211789.552239


In [518]:
X_val2.head(3)

,pred_test_lr,pred_test_svr,pred_test_dtr
0,232640.963887,163365.780077,209969.462957
1,161727.632251,163051.483418,143901.877139
2,237794.373398,163372.937236,228304.962890


In [519]:
model2 = Model2Linear()
pred_train_2, pred_val_2 = stacking(model2, X_train2, y_train, X_val2)
print ("MSE: {:.2f}".format(np.log(mean_squared_error(y_train, pred_train_2))))

MSE: 21.37


課題

・xgboostやLightGBMを使用したらもう少し精度が上がるのでは？

・パラメーター調整するともう少し精度が良くなるのでは？